## BERT Summary Extractor

in this notebook we load a pretrained BERT model, then use its encodings to evaluate wether a sentence is important or not.

The usual GOTO would be to append a classifier layer at the end of BERT to get our score, but nothing prevents us from computing the loss directly on the embedding (faster experiments)

In [ ]:
%%capture
!pip install transformers
!pip install datasets
!pip install rouge_score

In [ ]:
from tqdm import tqdm
import gc, re

import torch, torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from datasets import load_dataset, load_metric
from transformers import BertModel, BertTokenizer
from transformers.optimization import AdamW

In [ ]:
encoder_max_length = 400
decoder_max_length = 75
batch_size         = 4
model_name         = 'bert-base-uncased'

In [ ]:
%%capture

# just to download stuff at the beginning, hiding the logs after
BertModel.from_pretrained(model_name)
load_dataset("cnn_dailymail", "3.0.0")
BertTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
"""
Uses https://huggingface.co/datasets/viewer/?dataset=cnn_dailymail and a tokenizer
to create the hot-encodings to feed to the NN
"""

class CNNDataset(Dataset):
  def __init__(self, mode='train', n_articles=10000):
    super().__init__()
    raw_data  = load_dataset("cnn_dailymail", "3.0.0", split=f"{mode}[:{n_articles}]")  # download dataset from huggingface
    self.tokenizer = BertTokenizer.from_pretrained(model_name)    # load tokenizer to hot-encode articles
    self.data = self.preprocess_data(raw_data)                                          # preprocess data (hot-encode article/highlights)

  def preprocess_data(self, raw_data):
    data = []
    punctuations = set(self.tokenizer(". , ; : ? !"))
    for i in range(len(raw_data)):

      # hot encode article
      hot_article = self.tokenizer(raw_data[i]['article'], padding="max_length", truncation=True, max_length=encoder_max_length, return_tensors="pt").input_ids[0]
      hot_high  = self.tokenizer(raw_data[i]['highlights'], padding="max_length", truncation=True, max_length=decoder_max_length, return_tensors="pt").input_ids[0]

      # n-grams of highlights
      high_2grams, words = set(), set()
      prec = hot_high[0].item()
      for tok in hot_high[1:]:
        tok = tok.item()
        high_2grams.add((prec, tok))
        words.add(tok)
        prec = tok

      # n-grams of article (find best sentences)
      labels = torch.zeros_like(hot_article).float()
      for i, tok in enumerate(hot_article[1:]):
        tok = tok.item()
        if (prec, tok) in high_2grams:
          labels[i+1] = 1.
        prec = tok

      # important sentences
      labels2 = torch.zeros_like(hot_article).float()
      sum, prev = 0, 0
      for i, tok in enumerate(hot_article):
        tok = tok.item()
        sum += (tok in words and not tok in punctuations)*0.4 + labels[i]
        if tok in punctuations:
          labels[prev:i+1] = sum / (i+1-prev)
          sum, prev = 0, i+1


      data.append( {
          'input_ids':hot_article,       # article
          'decoder_input_ids':hot_high,  # summary
          'labels':labels,                            # summary (excluding <pad> from loss)
          'labels2':labels2,                            # summary (excluding <pad> from loss)
          'attention_mask':self.tokenizer(raw_data[i]['article'], padding="max_length", truncation=True, max_length=encoder_max_length, return_tensors="pt").attention_mask[0],
          'decoder_attention_mask':self.tokenizer(raw_data[i]['highlights'], padding="max_length", truncation=True, max_length=decoder_max_length, return_tensors="pt").attention_mask[0]
      } )
    return data
  
  def __len__(self):
    return len(self.data)
  
  def __getitem__(self, idx):
    return self.data[idx]


In [ ]:
class ModelSummarizer(nn.Module):
  def __init__(self):
    super().__init__()
    self.bert = BertModel.from_pretrained(model_name).cuda()

  def forward(self, batch_data):
    x = self.bert(**batch_data)
    return x[0][:, :, :2]

In [ ]:
"""
Instantiate model, dataloader & optimizer
"""

# model trained on xetreme summarization task
model = ModelSummarizer().cuda()

# data
if 'train_data' not in dir():
  train_data = CNNDataset(mode='train', n_articles=10000)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, pin_memory=True)

# optimizer
params = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in ["bias", "LayerNorm.weight"])],
        "weight_decay": 1e-8,
    },
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in ["bias", "LayerNorm.weight"])],
        "weight_decay": 0.0,
    },
]
optim = torch.optim.AdamW(params, lr=3e-4)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Reusing dataset cnn_dailymail (/root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234

## Finetuning not working

--> I think that the model was pretrained with a particular loss, for this reason finetuning do not work properly (or we just lack computing power)

In [ ]:

model.train()
for epoch in range(3):
  tot_loss = []
  for data in tqdm(train_loader, postfix=True):

    # move to GPU
    for k in data:
      data[k] = data[k].cuda()

    # forward
    input = {k:v for k,v in data.items() if k in {'input_ids','attention_mask'}}
    logits = model(input)

    important_sentences = torch.stack((data['labels'], data['labels2']), dim=2)
    loss = nn.MSELoss()(logits, important_sentences)

    # backward
    loss.backward()
    optim.step()
    optim.zero_grad()
    tot_loss.append(loss.item())

    # print stats
    if len(tot_loss)%100==0: 
      print(f'loss e:{epoch} = {1000*sum(tot_loss)/len(tot_loss):.5f}')
  print(f'FINAL loss e:{epoch} = {1000*sum(tot_loss)/len(tot_loss):.5f}')
  tot_loss=[]

  4%|▍         | 100/2500 [01:07<26:51,  1.49it/sTrue]

loss e:0 = 108.38672


  8%|▊         | 200/2500 [02:15<25:49,  1.48it/sTrue]

loss e:0 = 91.03345


 12%|█▏        | 300/2500 [03:22<24:33,  1.49it/sTrue]

...

loss e:2 = 53.22148


 96%|█████████▌| 2400/2500 [26:50<01:07,  1.49it/sTrue]

loss e:2 = 53.32303


100%|██████████| 2500/2500 [27:57<00:00,  1.49it/sTrue]

loss e:2 = 53.15068
FINAL loss e:2 = 53.15068


In [ ]:
if 'test_data' not in dir():
  test_data = CNNDataset(mode='test', n_articles=1000)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=False, pin_memory=True)

predicted, real = [], []

model.eval()
with torch.no_grad():
  for batch in test_loader:

    # forward
    input = {k:v.cuda() for k,v in data.items() if k in {'input_ids','attention_mask'}}
    logits = model(input)
    logits = logits.sum(dim=2)[0]

    # score sentences
    punctuations = test_data.tokenizer.encode(". ; ? !")
    score, sum = [], 0
    for val, tok in zip(logits, data['input_ids'][0]):
      sum += val.item()
      if tok in punctuations:
        score.append(sum)
        sum = 0
    

    # convert hot-encode into text
    predicted_summary = test_data.tokenizer.decode(batch['input_ids'][0], skip_special_tokens=True)
    blocks = [0]
    for i, char in enumerate(predicted_summary):
      if char in {".",";","?","!"}:
        blocks.append(i)
    blocks.append(len(predicted_summary))

    # get top 3
    score = score[:len(blocks)-1]
    if len(score)<3:
      idx = list(range(len(score)))
    else:
      _, idx = torch.tensor(score).topk(3)

    final = ''
    for i in idx:
      i = int(i)
      final += predicted_summary[blocks[i]+2:blocks[i+1]]
    
    # save decoded summaries
    real_summary = test_data.tokenizer.decode(batch['decoder_input_ids'][0], skip_special_tokens=True)
    predicted.append(final)
    real.append(real_summary)

  rouge = load_metric('rouge')

Reusing dataset cnn_dailymail (/root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234)


Downloading:   0%|          | 0.00/2.17k [00:00<?, ?B/s]

In [ ]:
score = rouge.compute(predictions=predicted, references=real)

for k in score:
  print(k, score[k].mid)

print('\n', predicted[2], '\n\n', real[2])

rouge1 Score(precision=0.1936956623231523, recall=0.2970620887408911, fmeasure=0.2226695122536908)
rouge2 Score(precision=0.05014442975045013, recall=0.0796845203961831, fmeasure=0.05839132807868044)
rougeL Score(precision=0.13192943844870555, recall=0.20167446552470977, fmeasure=0.1510992744850564)
rougeLsum Score(precision=0.13196855563764165, recall=0.2014674954178118, fmeasure=0.15113328949771845)

 i asked him about the science behind climate change and public health and the message he wants the average american to take away, as well as how enforceable his action plan iswhile in lhe credits the clean air act with making americans " a lot " healthier, in addition to being able to " see the mountains in the background because they aren't covered in smog 

 " no challenge poses more of a public threat than climate change, " the president says. he credits the clean air act with making americans " a lot " healthier.
